In [1]:
import os
import pandas as pd
import zipfile
import shutil
import random
from datetime import datetime
import numpy as np
# Path to the GTFS zipped folders
gtfs_folders_path = 'inputs/GTFS_Baseline10may2023/'
# gtfs_folders_path = 'inputs/GTFS_test/'

import os


In [ ]:
#Inputs

freq_scales =  [ 0.5, 0.75, 1.5, 1.5, 1, 2, ]
speed_scales = [ 0.5, 1, 1.5,  1,  1, 2,  ]
# Da fare (o rifare tutti se lenght by stop times compromised - Check order every time
# freq_scales =  [ 3, 0.75, 2, 3, 0.5]
# speed_scales = [ 1, 0.75, 1, 3, 1  ]
#ORDER SHAPES AND STOP TIMES WHEN READ...OE CHECK PATHTRAVERSAL MATCH
n_min_trips = 5
day_of_week = 'friday'  # for example: 'monday', 'tuesday', etc.
specific_date = 20170922  # format: 'YYYYMMDD'

# def change_speed(time_series, speed_factor):
#     # Convert the time series from a timedelta string to timedelta objects
#     time_deltas = pd.to_timedelta(time_series)

#     # Initialize the list with the first time unchanged
#     adjusted_times = [time_deltas.iloc[0]]

#     # Iterate over the time delta objects
#     for i in range(1, len(time_deltas)):
#         # Calculate the interval in seconds and adjust by the speed factor
#         interval_seconds = (time_deltas.iloc[i] - time_deltas.iloc[i - 1]).total_seconds() / speed_factor
        
#         # Add the adjusted interval to the last adjusted time
#         adjusted_time = adjusted_times[i - 1] + pd.to_timedelta(interval_seconds, unit='s')
        
#         # Append the new time to the adjusted times list
#         adjusted_times.append(adjusted_time)

#     # Convert the adjusted times list back to the same format as the input series
#     adjusted_times_str = ["0 days " + str(adjusted_time).split(' days ')[-1] for adjusted_time in adjusted_times]

#     return pd.Series(adjusted_times_str)

def change_speed(time_series, speed_factor):
    """
    Adjusts time deltas in a series based on a speed factor.
    :param time_series: Series of timedelta strings.
    :param speed_factor: Factor to adjust the speed.
    :return: Series with adjusted timedelta strings.
    """
    # Convert to timedelta if not already a Series
    if not isinstance(time_series, pd.Series):
        time_series = pd.Series(time_series)

    time_deltas = pd.to_timedelta(time_series)
    adjusted_times = [time_deltas.iloc[0]]

    # Adjust each time delta based on speed factor
    for i in range(1, len(time_deltas)):
        interval = (time_deltas.iloc[i] - time_deltas.iloc[i - 1]).total_seconds() / speed_factor
        adjusted_times.append(adjusted_times[i - 1] + pd.to_timedelta(interval, unit='s'))

    # Format adjusted times to strings
    # adjusted_times_str = adjusted_times.apply(lambda x: str(x).split(' days ')[-1] if 'days' in str(x) else str(x))
    adjusted_times_series = pd.Series(adjusted_times)
    adjusted_times_str = adjusted_times_series.apply(lambda x: str(x).split(' days ')[-1] if 'days' in str(x) else str(x))
    # print(time_series, adjusted_times_str)
    return adjusted_times_str


def format_timedelta_to_string(timedelta_value):
    """
    Formats a timedelta object into a string in 'HH:MM:SS' format.

    :param timedelta_value: Timedelta object to format.
    :return: String representation of the timedelta in 'HH:MM:SS' format.
    """
    total_seconds = int(timedelta_value.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"


def standardize_dep_time(time_value):
    """
    Standardizes various time formats (string or numeric) into a uniform 'HH:MM:SS' format.
    Handles strings that represent time deltas and numeric values assumed to represent seconds.

    :param time_value: The time value to standardize, can be a string or a numeric type.
    :return: A string representing the standardized time in 'HH:MM:SS' format.
             Returns None if the format is incorrect or parsing fails.
    """
    if pd.isna(time_value) or time_value == ' ':
        return ''
    try:
        if isinstance(time_value, str):
            time_value_strp = time_value.strip()
            if time_value_strp:
                timedelta_value = pd.to_timedelta(time_value_strp)
                return format_timedelta_to_string(timedelta_value)
            else:
                print(f'ERROR: Issue processing time_value {time_value}:{time_value_strp}')
        elif isinstance(time_value, (int, float)):
            timedelta_value = pd.to_timedelta(time_value, unit='s')
            return format_timedelta_to_string(timedelta_value)
    except ValueError as e:
        print(f'ERROR: Issue processing time_value {time_value}: {e}')
    except Exception as e:
        print(f'ERROR: Unexpected issue processing time_value {time_value}: {e}')

    return ''

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculates the great-circle distance between two points on the Earth's surface.

    Uses the Haversine formula to calculate the distance between two points 
    specified by longitude and latitude in decimal degrees.

    :param lon1: Longitude of the first point in decimal degrees.
    :param lat1: Latitude of the first point in decimal degrees.
    :param lon2: Longitude of the second point in decimal degrees.
    :param lat2: Latitude of the second point in decimal degrees.
    :return: Distance between the two points in meters.
    """
    R = 6371000.0
    # Convert coordinates from decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    # Calculate differences in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    # Apply Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Calculate and return the distance
    distance = R * c
    return distance


def calculate_shape_dist_traveled(shapes):
    """
    Calculates the cumulative distance traveled along a path defined by shape points.

    This function takes a DataFrame of shape points, each with latitude and longitude coordinates, 
    and computes the cumulative distance traveled along the path they define. It uses the Haversine 
    formula to calculate the distance between consecutive points.

    :param shapes: DataFrame containing the shape points. Must include columns 'shape_id', 
                   'shape_pt_sequence', 'shape_pt_lon', and 'shape_pt_lat'.
    :return: DataFrame with an additional column 'shape_dist_traveled' indicating the cumulative 
             distance traveled to each point in the shape.
    """
    shapes = shapes.sort_values(by=['shape_id', 'shape_pt_sequence'])
    
    distances = haversine(shapes['shape_pt_lon'].shift(), shapes['shape_pt_lat'].shift(),
                          shapes['shape_pt_lon'], shapes['shape_pt_lat'])
    
    distances = distances.fillna(0)
    
    shapes['shape_dist_traveled'] = distances.cumsum()
    
    return shapes

def calculate_stop_dist_traveled(relevant_stops):
    """
    Calculates the cumulative distance traveled between bus stops.

    :param stops: DataFrame containing the bus stops. Must include columns 'stop_id', 
                  'stop_sequence', 'stop_lon', and 'stop_lat'.
    :return: DataFrame with an additional column 'stop_dist_traveled' indicating the cumulative 
             distance traveled to each stop.
    """
    distances = haversine(relevant_stops['stop_lon'].shift(), relevant_stops['stop_lat'].shift(),
                          relevant_stops['stop_lon'], relevant_stops['stop_lat'])
    
    distances = distances.fillna(0)

    stops['stop_dist_traveled'] = distances.cumsum()
    
    return stops

# def get_vmt(group, trips, shapes, stops, stop_ids):
#     """
#     Calculates the total Vehicle Miles Traveled (VMT) for a group of trips using shape data.

#     :param group: Group of trips with common attributes.
#     :param trips: DataFrame of trip data.
#     :param shapes: DataFrame of shape data with distance information.
#     :return: Total VMT for the group of trips.
#     """
#     if len(shapes) == 0:
#         return 0

#     tot_length = 0

#     shape_ids = trips['shape_id'][(trips['route_id'].isin(group['route_id']) & 
#                                   (trips['trip_headsign'].isin(group['trip_headsign'])))]
#     for shape_id in shape_ids:
#         relevant_shapes = shapes[shapes['shape_id'] == shape_id]
#         try:
#             shapes_with_distance = calculate_shape_dist_traveled(relevant_shapes)
#             dist = shapes_with_distance['shape_dist_traveled'].max()
#             tot_length += dist

#             if dist < 1 or dist > 100000:  
#                 print(f'WARNING: Unusual distance for shape {shape_id}: {dist} meters')

#         except Exception as e:
#             print(f'ERROR: Issue processing shape {shape_id}: {e}')
#             try:
#                 relevant_stops = stops[stops['stop_id'].isin(stop_ids)]
#                 stops_with_distance = calculate_stop_dist_traveled(relevant_stops)
#                 dist = stops_with_distance['shape_dist_traveled'].max()
#                 tot_length += dist

#                 if dist < 1 or dist > 100000:  
#                     print(f'WARNING: Unusual distance for shape {shape_id}: {dist} meters')
#             except Exception as e:
#                 print(f'ERROR: Issue processing shape {shape_id}: {e}')

            

#     return tot_length

def get_vmt2(trips, trip_id, shapes, stops, stop_ids):
    """
    Get VMT for a single trip
    """
    
    tot_length = 0
    
    if len(shapes) == 0:
        try:
            relevant_stops = pd.DataFrame(stop_ids, columns=['stop_id']).merge(stops, on='stop_id', how='left')
            if len(relevant_stops) != len(stop_ids):
                print(f'ERROR: Issue merging stops {stop_ids}')
            stops_with_distance = calculate_stop_dist_traveled(relevant_stops)
            dist = stops_with_distance['stop_dist_traveled'].max()
            tot_length += dist

            if dist < 1 or dist > 100000:  
                print(f'WARNING: Unusual distance for stops {stop_ids}: {dist} meters')
        except Exception as e:
            print(f'ERROR: Issue processing stops {stop_ids}: {e}')
                    
    else:
        shape_id = list(trips['shape_id'][(trips['trip_id'] == trip_id)])[0]
        relevant_shapes = shapes[shapes['shape_id'] == shape_id]

        try:
            shapes_with_distance = calculate_shape_dist_traveled(relevant_shapes)
            dist = shapes_with_distance['shape_dist_traveled'].max()
            tot_length = dist

            if dist < 1 or dist > 100000:  
                print(f'WARNING: Unusual distance for shape {shape_id}: {dist} meters')
  
        except Exception as e:
            print(f'ERROR: Issue processing shape {shape_id}: {e}')


    return tot_length

# def get_vht(group, trips, stop_times):
#     """
#     Calculates the total Vehicle Hours Traveled (VHT) for a group of trips.

#     :param group: Group of trips with common attributes.
#     :param trips: DataFrame of trip data.
#     :param stop_times: DataFrame of stop times for the trips.
#     :return: Total VHT for the group of trips.
#     """
#     tot_dur = 0
#     trip_ids = trips['trip_id'][(trips['route_id'].isin(group['route_id']) & (trips['trip_headsign'].isin(group['trip_headsign'])))]

#     for trip_id in trip_ids:
#         relevant_stop_times = stop_times[stop_times['trip_id'] == trip_id]
#         try:
#             departure_times = pd.to_timedelta(relevant_stop_times['departure_time'])
#             if len(departure_times) >= 2:
#                 diff = (departure_times.iloc[-1] - departure_times.iloc[0]).total_seconds()
#                 tot_dur += diff
#                 if diff < 60 or diff > 10800:  
#                     print(f'WARNING: Unusual duration for trip {trip_id}: {diff} seconds')
#             else:
#                 print(f'WARNING: Not enough departure times for trip {trip_id}')
#         except ValueError as e:
#             print(f'ERROR: Problem processing times for trip {trip_id}: {e}')
#         except IndexError as e:
#             print(f'ERROR: Index error for trip {trip_id}: {e}')

#     return tot_dur

def get_vht2(trips, trip_id, stop_times):
    """
    Get VHT for a single trip
    """
    relevant_stop_times = stop_times[stop_times['trip_id'] == trip_id]
    try:
        departure_times = pd.to_timedelta(relevant_stop_times['departure_time'])
        if len(departure_times) >= 2:
            diff = (departure_times.iloc[-1] - departure_times.iloc[0]).total_seconds()
            tot_dur = diff
            if diff < 60 or diff > 10800:  
                print(f'WARNING: Unusual duration for trip {trip_id}: {diff} seconds')
        else:
            print(f'WARNING: Not enough departure times for trip {trip_id}')
    except ValueError as e:
        print(f'ERROR: Problem processing times for trip {trip_id}: {e}')
    except IndexError as e:
        print(f'ERROR: Index error for trip {trip_id}: {e}')

    return tot_dur


def create_freq_directory(gtfs_folders_path, freq_scale, speed_scale):
    """
    Creates a directory for storing GTFS data with specified frequency and speed scale.

    :param gtfs_folders_path: Base path where the new directory will be created.
    :param freq_scale: Frequency scaling factor to be included in the directory name.
    :param speed_scale: Speed scaling factor to be included in the directory name.
    :return: Path to the created directory.
    """
    freq_dir_name = f"GTFS_freq_{freq_scale}_speed_{speed_scale}"
    freq_dir_path = os.path.join(gtfs_folders_path, freq_dir_name)
    os.makedirs(freq_dir_path, exist_ok=True)
    return freq_dir_path

def save_dataframe_to_csv(df, temp_dir, filename):
    """
    Saves a DataFrame to a CSV file in the specified directory.

    :param df: DataFrame to save.
    :param temp_dir: Directory where the CSV file will be saved.
    :param filename: Name of the CSV file.
    """
    file_path = os.path.join(temp_dir, filename)
    df.to_csv(file_path, index=False)

def zip_directory(input_dir, output_zip):
    """
    Compresses an entire directory into a ZIP file.

    :param input_dir: Directory to compress.
    :param output_zip: Path to the output ZIP file.
    """
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(input_dir):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, input_dir))

def cleanup_temp_dir(temp_dir):
    """
    Removes a temporary directory and all its contents.

    :param temp_dir: Path to the temporary directory to remove.
    """
    shutil.rmtree(temp_dir)

def has_time_greater_than_24(group):
    """
    Checks if any time in a group exceeds 24 hours or is less than or equal to 4 hours.

    :param group: DataFrame group with the 'departure_time' column.
    :return: True if any time exceeds 24 hours or is <= 4 hours, False otherwise.
    """
    for time_str in group['departure_time']:
        if isinstance(time_str, str) and time_str.strip():
            try:
                time_delta = pd.to_timedelta(time_str)
                hours = time_delta.total_seconds() // 3600
                if hours >= 24 or hours <= 4:
                    return True
            except ValueError:
                return False
    return False

def get_dep_times(group, stop_times):
    """
    Extracts the earliest departure times for each trip in a group.

    :param group: DataFrame group of trips.
    :param stop_times: DataFrame of stop times.
    :return: List of earliest departure times for each trip.
    """
    departing_times = []
    for index, row in group.iterrows():
        trip_id = row['trip_id']
        stop_times_trip = stop_times[stop_times.trip_id == trip_id]
        departing_times.append(min(stop_times_trip.departure_time))
    # print('ddd',departing_times)
    return [datetime.strptime(time, '%H:%M:%S') for time in departing_times]

def get_dep_times2(trip_id, stop_times_trip):
    """
    Extracts the earliest departure times for a trip.

    :param group: DataFrame group of trips.
    :param stop_times: DataFrame of stop times.
    :return: List of earliest departure times for each trip.
    """
    departing_time = min(stop_times_trip.departure_time)
    return datetime.strptime(departing_time, '%H:%M:%S')

def get_data(zip_file, day_of_week, specific_date):
    """
    Extracts GTFS data from a ZIP file and filters trips based on the specified day and date.

    :param zip_file: Path to the ZIP file containing GTFS data.
    :param day_of_week: Day of the week to filter trips (e.g., 'monday', 'tuesday', etc.).
    :param specific_date: Specific date to filter trips in 'YYYYMMDD' format.
    :return: Tuple containing DataFrames for trips, stop_times, shapes, routes, 
             grouped_trips, and the path to the temporary directory.
    """
    if zip_file.endswith('.zip'):
        zip_path = os.path.join(gtfs_folders_path, zip_file)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            temp_dir = os.path.join(gtfs_folders_path, zip_file.split('.')[0])
            os.makedirs(temp_dir, exist_ok=True)
            zip_ref.extractall(temp_dir)

            trips = pd.read_csv(os.path.join(temp_dir, 'trips.txt'), dtype={'route_id': str, 'service_id': str, 'trip_id': str, 'shape_id': str})
            stop_times = pd.read_csv(os.path.join(temp_dir, 'stop_times.txt'), dtype={'trip_id': str,'stop_id': str})
            routes = pd.read_csv(os.path.join(temp_dir, 'routes.txt'), dtype={'route_id': str})
            stops = pd.read_csv(os.path.join(temp_dir, 'stops.txt'), dtype={'stop_id': str})
            stop_times = stop_times.sort_values(by=['trip_id', 'stop_sequence'])
            
            
            try:
                shapes = pd.read_csv(os.path.join(temp_dir, 'shapes.txt'), dtype={'shape_id': str})
                shapes = shapes.sort_values(by=['shape_id', 'shape_pt_sequence'])
            except:
                shapes = []
                print('WARNING: no shapes for', temp_dir)
                
            len_stop_times = len(stop_times) 
            stop_times['departure_time'] = stop_times['departure_time'].apply(standardize_dep_time)
            stop_times = stop_times[stop_times['departure_time'] != '']  
            stop_times = stop_times.dropna(subset=['departure_time'])
            if len(stop_times)<0.85*len_stop_times:
                print('WARNING:',  len_stop_times, 'len stop_times pre', len(stop_times), 'len stop_times after standardize')
            
            len_stop_times = len(stop_times)
            stop_times = stop_times.groupby('trip_id').filter(lambda x: not has_time_greater_than_24(x))
            if len(stop_times)<len_stop_times*0.85:
                print('WARNING:',len(stop_times), 'len stop_times pre',len(stop_times), 'len stop_times after >24')           
            
            len_trips = len(trips)
            calendar = pd.read_csv(os.path.join(temp_dir, 'calendar.txt'), dtype={'service_id': str})
            active_services = list(calendar['service_id'][(calendar[day_of_week] == 1)&
                                       (calendar['start_date'].astype(int)<specific_date)&
                                       (calendar['end_date'].astype(int)>specific_date)])
            print('active_services',active_services)
            print('trips[service_id]',trips['service_id'])
            trips = trips[trips['service_id'].isin(active_services)]
            if len_trips*0.85>len(trips):
                print('WARNING:',len_trips, 'len trips pre',len(trips), 'len trips after Calendar')
            
            len_stop_times = len(stop_times)
            stop_times = stop_times[stop_times.trip_id.isin(list(trips['trip_id']))]
            if len(stop_times)<len_stop_times*0.85:
                print('WARNING:',len(stop_times), 'len stop_times pre',len(stop_times), 'len stop_times trip_id in trips',)  
                
            len_trips = len(trips)
            trips = trips[trips['trip_id'].isin(list(stop_times['trip_id']))]
            if len_trips*0.85>len(trips):
                print('WARNING:',len_trips, 'len trips pre',len(trips), 'len trips after trip_id in stop_times')
            
#             try:
#                 len_shapes = len(shapes)
#                 shapes = shapes[shapes['shape_id'].isin(list(trips['shape_id']))]
#                 if len_shapes*0.85>len(shapes):
#                     print('WARNING:',len_shapes, 'len shapes pre',len(shapes), 'len shapes after shape_id in shapes')
#             except:
#                 print('WARNING: no shapes in ', zip_file)
            
#             len_routes = len(routes)
#             routes = routes[routes['route_id'].isin(list(trips['route_id']))]
#             if len_routes*0.85>len(routes):
#                 print('WARNING:',len_routes, 'len routes pre',len(routes), 'len routes after route_id in routes')
            
#             len_stops = len(stops)
#             stops = stops[stops['stop_id'].isin(list(stop_times['stop_id']))]
#             if len_stops*0.85>len(stops):
#                 print('WARNING:',len_stops, 'len stops pre',len(stops), 'len stops after stop_id in routes')
            
            
            grouped_trips = trips.groupby(['route_id','trip_headsign'])
        
        return trips, stop_times, shapes, routes, grouped_trips, temp_dir, stops
    else:
        return [],[],[], [],[],[], []

def change_frequency_speed(group, trips, stop_times, departing_times, min_dep_time, max_dep_time, n_current_trips, n_new_trips):
    """
    Adjusts the frequency and speed of trips in a GTFS dataset.

    This function modifies the frequency of trips by adding or removing trips based on the desired new frequency
    and optionally adjusts the speed of each trip. 

    :param group: Grouped DataFrame of trips with common attributes.
    :param trips: DataFrame containing trip data.
    :param stop_times: DataFrame containing stop times data.
    :param departing_times: List of earliest departure times for each trip.
    :param min_dep_time: Minimum departure time across all trips.
    :param max_dep_time: Maximum departure time across all trips.
    :param n_current_trips: Current number of trips.
    :param n_new_trips: Desired number of trips after adjustment.
    :return: Modified trips and stop_times DataFrames and the new frequency.
    """
    n_trips_to_add = n_new_trips-n_current_trips
    if n_trips_to_add>=0:
        new_trip_ids = random.choices(range(0, n_current_trips - 1), k=n_trips_to_add)
    elif  n_trips_to_add<0:
        new_trip_ids = random.sample(range(0, n_current_trips - 1), n_new_trips)
    new_trip_ids.sort()
    new_frequency = (max_dep_time-min_dep_time)/n_new_trips


    dep_time = min_dep_time
    i = 0
    for (index, row), pre_dep_time in zip(group.iterrows(), departing_times):
        trip_id = row['trip_id']
        if n_trips_to_add>=0:
            n_duplicates = 0
            for trip_index in new_trip_ids:
                if trip_index == i:
                    n_duplicates+=1

            if n_duplicates>0:
                for j in range(n_duplicates):
                    index_to_duplicate = trips[trips['trip_id'] == trip_id].index[0]
                    trip_to_duplicate = trips.loc[index_to_duplicate].copy()
                    trip_to_duplicate['trip_id'] = str(trip_id)+'_dupli_'+str(j)
                    trips = trips.append(trip_to_duplicate, ignore_index=True)
                    stop_times_to_duplicate = stop_times[stop_times['trip_id'] == trip_id].copy()
                    stop_times_to_duplicate['trip_id'] = str(trip_id)+'_dupli_'+str(j)
                    time_diff = dep_time-pre_dep_time
                    # print('7 - time_diff = dep_time-pre_dep_time', time_diff,dep_time, pre_dep_time)
                    stop_times_to_duplicate['departure_time'] += time_diff
                    # print('8 - stop_times_to_duplicate[departure_time] += time_diff', stop_times_to_duplicate['departure_time'])
                    # if speed_scale != 1:
                    stop_times_to_duplicate['departure_time'] = list(change_speed(stop_times_to_duplicate['departure_time'], speed_scale))
                    # print('9 - after changing speed', stop_times_to_duplicate['departure_time'])
                    stop_times_to_duplicate['arrival_time'] = stop_times_to_duplicate['departure_time']
                    stop_times = pd.concat([stop_times, stop_times_to_duplicate], ignore_index=True)
                    dep_time+=new_frequency
                    # print('10 - dep_time+=new_frequency', dep_time,new_frequency)
                    # print('11 - stop_times = pd.concat([stop_times, stop_times_to_duplicate]',stop_times['departure_time'])

            time_diff =  dep_time-pre_dep_time
            index_to_update = stop_times['trip_id'] == trip_id
            stop_times.loc[index_to_update, 'departure_time'] = stop_times.loc[index_to_update, 'departure_time'] + time_diff
            # if speed_scale != 1:
            stop_times.loc[index_to_update, 'departure_time'] = list(change_speed(stop_times.loc[index_to_update, 'departure_time'], speed_scale))
            stop_times.loc[index_to_update, 'arrival_time'] = stop_times.loc[index_to_update, 'departure_time'] 
            dep_time+=new_frequency
            # print('eee',stop_times['departure_time'])
            # print('fff',dep_time)
            
        elif  n_trips_to_add<0:
            if i in new_trip_ids:
                time_diff =  dep_time-pre_dep_time
                index_to_update = stop_times['trip_id'] == trip_id
                # print('kkk',stop_times.loc[index_to_update, 'departure_time'])
                stop_times.loc[index_to_update, 'departure_time'] = stop_times.loc[index_to_update, 'departure_time'] + time_diff
                # print('jjj',stop_times.loc[index_to_update, 'departure_time'])
                # if speed_scale != 1:
                stop_times.loc[index_to_update, 'departure_time'] = list(change_speed(stop_times.loc[index_to_update, 'departure_time'], speed_scale))
                stop_times.loc[index_to_update, 'arrival_time'] = stop_times.loc[index_to_update, 'departure_time'] 
                # print('hhh',stop_times.loc[index_to_update, 'departure_time'])
                dep_time+=new_frequency
                # print('ggg',dep_time)
                # print('iii',time_diff)
                
            else:
                trips = trips[trips['trip_id']!=trip_id]
        i+=1
    
    len_stop_times = len(stop_times)
    stop_times = stop_times.groupby('trip_id').filter(lambda x: not has_time_greater_than_24(x))
    len_trips = len(trips)
    trips = trips[trips.trip_id.isin(list(stop_times['trip_id']))]
    stop_times = stop_times[stop_times.trip_id.isin(list(trips['trip_id']))]
    if len_stop_times*0.85>len(stop_times):
        print('WARNING:',len_stop_times, 'len stop_times pre',len(stop_times), 'len stop_times after >24_2',len_trips, 'len trips pre',len(trips), 'len trips after >24_2')

    return trips, stop_times, new_frequency
    
def create_trip_tables(agency, trips, stop_times, shapes, routes, stops):
    
    # Initialize a dictionary to store tables for each trip
    # trip_tables = {}
    all_trips_data = []
    for trip_id, stop_sign, route_id in zip(trips['trip_id'], trips['trip_headsign'],  trips['route_id']):
        involved_stop_times = stop_times[stop_times['trip_id']==trip_id]
        involved_routes = routes[routes['route_id']==route_id]
        stop_ids = list(involved_stop_times['stop_id'])
        if len(shapes)>0:
            shape_id = list(trips[trips['trip_id']==trip_id]['shape_id'])[0]

            shape_pt_lat = list(shapes[shapes['shape_id']==shape_id]['shape_pt_lat'])
            shape_pt_lon = list(shapes[shapes['shape_id']==shape_id]['shape_pt_lon'])
        else:
            shape_pt_lat = list(pd.DataFrame(stop_ids, columns=['stop_id']).merge(stops, on='stop_id', how='left')['stop_lat'])
            shape_pt_lon = list(pd.DataFrame(stop_ids, columns=['stop_id']).merge(stops, on='stop_id', how='left')['stop_lon'])
            if len(shape_pt_lat) != len(stop_ids) | len(shape_pt_lon) != len(stop_ids):
                print(f'ERROR: Issue merging stops {stop_ids}')
            # shape_pt_lat = list(stops[stops['stop_id'].isin(stop_ids)]['stop_lat'])
            # shape_pt_lon = list(stops[stops['stop_id'].isin(stop_ids)]['stop_lon'])
        # Initialize summary dictionary for each trip
        trip_data = {
            'agency': agency,
            'route_id': route_id,
            'route_mode': list(involved_routes['route_type'])[0],
            'route_name': list(involved_routes['route_short_name'])[0],
            'route_long_name': list(involved_routes['route_long_name'])[0],
            # 'route_descr': list(involved_routes['route_desc'])[0],
            'trip_id': trip_id,
            'trip_headsign': stop_sign,
            'departure_time': get_dep_times2(trip_id, involved_stop_times),
            'lenght': get_vmt2(trips, trip_id, shapes, stops, stop_ids),
            'duration': get_vht2(trips, trip_id, stop_times),
            'shape_list_lat': shape_pt_lat,
            'shape_list_lon': shape_pt_lon,
            # Add additional calculations and fields as necessary
            # 'VMT': calculate_vmt(group),
            # 'VHT': calculate_vht(group),
        }
        all_trips_data.append(trip_data)
        # Convert summary dictionary to DataFrame
    trip_tables = pd.DataFrame(all_trips_data)

    return trip_tables

for freq_scale, speed_scale in zip(freq_scales, speed_scales):
    
    freq_dir_path = create_freq_directory(gtfs_folders_path, freq_scale, speed_scale)


    trip_tables = pd.DataFrame()
    trip_tables_pre = pd.DataFrame()
    
    for zip_file in os.listdir(gtfs_folders_path):
        if zip_file.endswith('.zip'):

            agency = zip_file[:2]
            print('Get Data...', agency)
            
            #get_data
            trips, stop_times, shapes, routes, grouped_trips, temp_dir, stops = get_data(zip_file,day_of_week, specific_date)
            # print('1 - after getting data, departing times of stop times', stop_times['departure_time'])
            trip_tables_pre = pd.concat([trip_tables_pre, create_trip_tables(agency, trips, stop_times, shapes, routes, stops)])
            trip_tables_pre.to_csv(f'outputs/TripTablePre_GTFS_freq_{freq_scale}_speed_{speed_scale}.csv')

            if len(grouped_trips)>0:
                print(zip_file,len(grouped_trips))
                for name, group in grouped_trips:
                    
                    print('################################')
                    print('name', name)

                    departing_times = get_dep_times(group , stop_times)
                    group['departing_time'] = departing_times

                    # Sort the group based on departing times
                    group_sorted = group.sort_values(by='departing_time')
                    
                    # departing_times.sort()
                    # print('2 - Get departing times for the group', departing_times)
                    min_dep_time = min(departing_times)
                    max_dep_time = max(departing_times)
                    # print('3 - Min and max', min_dep_time, max_dep_time)
                    duration = max_dep_time - min_dep_time
                    # print('4 - Duration', duration)
                    duration_in_seconds = duration.total_seconds()
                    # print('5 - Duration to seconds', duration_in_seconds)

                    if len(group)>n_min_trips:

                        n_current_trips = len(departing_times)
                        n_new_trips = int(freq_scale*n_current_trips)
                        print('n_new_trips', n_new_trips)
                        len_trips_before = len(trips)
                        
                        #change_frequency_speed
                        trips, stop_times, new_frequency = change_frequency_speed(group, trips, stop_times,departing_times, min_dep_time, max_dep_time, n_current_trips, n_new_trips)


                # stop_times['departure_time'] = pd.to_timedelta(stop_times['departure_time'])
                # stop_times['departure_time'] = stop_times['departure_time'].dt.components.apply(
                #     lambda x: f"{int(x.hours):02d}:{int(x.minutes):02d}:{int(x.seconds):02d}", axis=1)
                # stop_times['arrival_time'] = pd.to_timedelta(stop_times['arrival_time'])
                # stop_times['arrival_time'] = stop_times['arrival_time'].dt.components.apply(
                #     lambda x: f"{int(x.hours):02d}:{int(x.minutes):02d}:{int(x.seconds):02d}", axis=1)
                len_stop_times = len(stop_times)
                stop_times['departure_time'] = stop_times['departure_time'].apply(standardize_dep_time)
                stop_times['arrival_time'] = stop_times['arrival_time'].apply(standardize_dep_time)
                stop_times = stop_times[stop_times['departure_time'] != '']  
                stop_times = stop_times.dropna(subset=['departure_time'])
                if len(stop_times)<len_stop_times:
                    print('ERROR:',  len_stop_times, 'len stop_times pre', len(stop_times), 'len stop_times after SECOND standardize')
                
                # print('STANDARD departure_time',stop_times['departure_time'])
                # print('STANDARD arrival_time',stop_times['arrival_time'])
                
                save_dataframe_to_csv(trips, temp_dir, 'trips.txt')
                save_dataframe_to_csv(stop_times, temp_dir, 'stop_times.txt')
                # save_dataframe_to_csv(stops, temp_dir, 'stops.txt')
                # save_dataframe_to_csv(routes, temp_dir, 'routes.txt')
                # try:
                #     save_dataframe_to_csv(shapes, temp_dir, 'shapes.txt')
                # except:
                #     print('Warning, no shapes for', agency)

                trip_tables = pd.concat([trip_tables, create_trip_tables(agency, trips, stop_times, shapes, routes, stops)])
                trip_tables.to_csv(f'outputs/TripTable_GTFS_freq_{freq_scale}_speed_{speed_scale}.csv')
                
                #Save outputs
                output_zip = os.path.join(freq_dir_path, zip_file)
                zip_directory(temp_dir, output_zip)
                cleanup_temp_dir(temp_dir)


            else:
                print('WARNING: agency without trips', agency)
                  
                  



    trip_tables.to_csv(f'outputs/TripTable_GTFS_freq_{freq_scale}_speed_{speed_scale}.csv')
    trip_tables_pre.to_csv(f'outputs/TripTablePre_GTFS_freq_{freq_scale}_speed_{speed_scale}.csv')



Get Data... FF
active_services ['c_70_b_207_d_31', 'c_2557_b_230_d_31', 'c_2557_b_230_d_16']
trips[service_id] 0      c_2557_b_230_d_15
1      c_2557_b_230_d_15
2      c_2557_b_230_d_15
3      c_2557_b_230_d_15
4      c_2557_b_230_d_16
             ...        
729      c_70_b_207_d_32
730      c_70_b_207_d_32
731      c_70_b_207_d_32
732      c_70_b_207_d_32
733      c_70_b_207_d_32
Name: service_id, Length: 734, dtype: object
FF.zip 32
################################
name ('190', 'Davis Sacramento')
################################
name ('190', 'Davis/Fairfield')
################################
name ('190', 'Dixon')
################################
name ('190', 'Downtown Sacramento')
################################
name ('190', 'Fairfield')
################################
name ('190', 'Vacaville Davis')
################################
name ('190', 'Vacaville Fairfield')
################################
name ('193', 'Fairfield Vacaville')
n_new_trips 6
############################

/tmp/ipykernel_505034/726011659.py:533: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trips = trips.append(trip_to_duplicate, ignore_index=True)
/tmp/ipykernel_505034/726011659.py:533: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trips = trips.append(trip_to_duplicate, ignore_index=True)


################################
name ('196', 'WC PH BART')
n_new_trips 18
################################
name ('259', 'El Cerrito Del Norte BART Station AC Transit')
n_new_trips 45


/tmp/ipykernel_505034/726011659.py:533: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trips = trips.append(trip_to_duplicate, ignore_index=True)
/tmp/ipykernel_505034/726011659.py:533: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trips = trips.append(trip_to_duplicate, ignore_index=True)


################################
name ('259', 'FTC')
n_new_trips 25
################################
name ('259', 'Suisun City Amtrak')
n_new_trips 21


...

################################
name ('40-117', '40 - Davenport - North Coast Beaches')
################################
name ('40-118', '40 - Davenport - North Coast Beaches')
n_new_trips 32


/tmp/ipykernel_505034/726011659.py:533: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trips = trips.append(trip_to_duplicate, ignore_index=True)


################################
name ('41-117', '41 - Bonny Doon')
################################
name ('41-118', '41 - Bonny Doon')
n_new_trips 32


/tmp/ipykernel_505034/726011659.py:533: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trips = trips.append(trip_to_duplicate, ignore_index=True)


################################
name ('42-117', '42 - Davenport - Bonny Doon')
################################
name ('42-118', '42 - Davenport - Bonny Doon')
n_new_trips 48


### ***Aggregate Transit Trip Table***

### ***Add Sim results: ridership, shifted from cars, av income, estimated new vehicles***

### ***Analyze economical and accessibility and mmode shift cases***